# Get result score notebook

Given a result.csv dataset we have the MRR associated to the given algorithm

In [1]:
import math
import pandas as pd
import numpy as np
import csv
import pandasql as ps

In [2]:
result_csv = pd.read_csv("~/Documents/Tesi/trivagoRecSysChallengeData2019_v2/prova1.csv")

In [3]:
gt_csv = pd.read_csv("~/Documents/Tesi/trivagoRecSysChallengeData2019_v2/validation.csv")

In [6]:
result_csv.head()

,user_id,session_id,timestamp,step,item_recommendations
0,0004Q49X39PY,9de47d9a66494,1541641157,1,2213014 3184842 2292254 3812004 8153310 320289...
1,0004Q49X39PY,beea5c27030cb,1541561202,1,3812004 3505150 2292254 3202894 4476010 710135...
2,00071784XQ6B,9617600e1ba7c,1541630328,2,22721 22854 16121 22819 3067559 22764 1478189 ...
3,0008BO33KUQ0,2d0e2102ee0dc,1541636411,6,507861 2176280 1669587 4272108 3133204 502066 ...
4,000GO9NY6P4M,55dbafdbb9bab,1541594662,2,157710 1287958 160577 319866 483691 1618677 40...


In [8]:
def read_into_df(file):
    """Read csv file into data frame."""
    df = (
        pd.read_csv(file)
            .set_index(['user_id', 'session_id', 'timestamp', 'step'])
    )

    return df


def generate_rranks_range(start, end):
    """Generate reciprocal ranks for a given list length."""

    return 1.0 / (np.arange(start, end) + 1)


def convert_string_to_list(df, col, new_col):
    """Convert column from string to list format."""
    fxn = lambda arr_string: [int(item) for item in str(arr_string).split(" ")]

    mask = ~(df[col].isnull())

    df[new_col] = df[col]
    df.loc[mask, new_col] = df[mask][col].map(fxn)

    return df


def get_reciprocal_ranks(ps):
    """Calculate reciprocal ranks for recommendations."""
    mask = ps.reference == np.array(ps.item_recommendations)

    if mask.sum() == 1:
        rranks = generate_rranks_range(0, len(ps.item_recommendations))
        return np.array(rranks)[mask].min()
    else:
        return 0.0


def score_submissions(subm_csv, gt_csv, objective_function):
    """Score submissions with given objective function."""

    print(f"Reading ground truth data {gt_csv} ...")
    df_gt = read_into_df(gt_csv)

    print(f"Reading submission data {subm_csv} ...")
    df_subm = read_into_df(subm_csv)

    # create dataframe containing the ground truth to target rows
    cols = ['reference', 'impressions', 'prices']
    df_key = df_gt.loc[:, cols]

    # append key to submission file
    df_subm_with_key = df_key.join(df_subm, how='inner')
    df_subm_with_key.reference = df_subm_with_key.reference.astype(int)
    df_subm_with_key = convert_string_to_list(
        df_subm_with_key, 'item_recommendations', 'item_recommendations'
    )

    # score each row
    df_subm_with_key['score'] = df_subm_with_key.apply(objective_function, axis=1)
    mrr = df_subm_with_key.score.mean()

    return mrr

In [15]:
SUBMISSION_COLUMNS = {'user_id', 'session_id', 'timestamp',
                      'step', 'item_recommendations'}


def get_test_session_set(df_test):
    """Identify sessions for which predictions have to be made."""

    mask = df_test.reference.isnull() & (df_test.action_type == 'clickout item')
    test_session_set = set(df_test[mask]['session_id'])

    return test_session_set


def check_passed(check=True):
    """Print result of check."""

    if check:
        print('> check passed')
    else:
        print('x check failed')


def check_duplicates(df):
    """Check if there are duplicate sessions in the df."""

    check_dupl = (len(df['session_id']) == len(df['session_id'].unique()))

    return check_dupl


def check_columns(df):
    """Check if the submission has the correct column names."""

    check_cols = (set(df.columns) == SUBMISSION_COLUMNS)

    return check_cols


def check_sessions(df_subm, df_test):
    """Check if the submission contains the correct sessions."""

    set_test_sessions = get_test_session_set(df_test)
    set_subm_sessions = set(df_subm['session_id'])
    check_sess = (set_test_sessions == set_subm_sessions)

    return check_sess

In [26]:
def get_submission_target(df):
    """Identify target rows with missing click outs."""

    mask = df["reference"].isnull() & (df["action_type"] == "clickout item")
    df_out = df[mask]

    return df_out

In [22]:
check_cols = check_columns(result_csv)
check_passed(check_cols)

> check passed


In [23]:
check_dupl = check_duplicates(result_csv)
check_passed(check_dupl)

> check passed


In [32]:
check_sess = check_sessions(result_csv, gt_csv)
check_passed(check_sess)

x check failed


In [61]:
mrr = score_submissions('~/Documents/Tesi/trivagoRecSysChallengeData2019_v2/prova1.csv', 
                        "~/Documents/Tesi/trivagoRecSysChallengeData2019_v2/validation.csv", get_reciprocal_ranks)
mrr

Reading ground truth data ~/Documents/Tesi/trivagoRecSysChallengeData2019_v2/validation.csv ...
Reading submission data ~/Documents/Tesi/trivagoRecSysChallengeData2019_v2/prova1.csv ...


0.5082995322768837

In [63]:
mrr = score_submissions("~/Documents/Tesi/trivagoRecSysChallengeData2019_v2/formula_interaction03.csv", 
                        "~/Documents/Tesi/trivagoRecSysChallengeData2019_v2/validation.csv", get_reciprocal_ranks)
mrr

Reading ground truth data ~/Documents/Tesi/trivagoRecSysChallengeData2019_v2/validation.csv ...
Reading submission data ~/Documents/Tesi/trivagoRecSysChallengeData2019_v2/formula_interaction03.csv ...


0.5493158022778003

The `my_formula` algorithm gets 0.549 MRR